# PydanticAI fundamentals

In [5]:
from pydantic_ai import Agent

agent = Agent(model="google-gla:gemini-2.5-flash")

result = await agent.run("Give me an IT employee working in Sweden, no yapping")

result

AgentRunResult(output='**Name:** Markus Svensson\n**Age:** 35\n**Role:** Senior Cloud Architect\n**Location:** Gothenburg, Sweden\n**Company:** Nordex AB (major telecom solutions provider)\n**Focus:** Designing and implementing scalable cloud infrastructure (Azure, AWS), containerization (Kubernetes), ensuring security and compliance for critical systems.')

In [6]:
print(result.output)

**Name:** Markus Svensson
**Age:** 35
**Role:** Senior Cloud Architect
**Location:** Gothenburg, Sweden
**Company:** Nordex AB (major telecom solutions provider)
**Focus:** Designing and implementing scalable cloud infrastructure (Azure, AWS), containerization (Kubernetes), ensuring security and compliance for critical systems.


## use pydantic to structure output

In [8]:
from pydantic import BaseModel, Field


class Employee(BaseModel):
    name: str = Field(description="first name and last name of the person")
    age: int = Field(gt=17, lt=67, description="age of a person")
    salary: int = Field(
        gt=30_000,
        lt=70_000,
        description="salary of the employee, the more senior the role, the higher salary",
    )
    role: str = Field(
        description="always include seniority level in the role (junior, senior, expert)"
    )
    description: str = Field(
        description="short description (max 2 sentences) of the role, include keywords"
    )


result = await agent.run(
    "Give me an IT employee working as data engineer in Sweden", output_type=Employee
)

result

AgentRunResult(output=Employee(name='Kalle Johansson', age=35, salary=60000, role='Senior Data Engineer', description='Designs, builds, and maintains robust data pipelines and ETL processes. Focuses on large-scale data processing, data warehousing, and big data technologies.'))

In [10]:
employee = result.output
employee

Employee(name='Kalle Johansson', age=35, salary=60000, role='Senior Data Engineer', description='Designs, builds, and maintains robust data pipelines and ETL processes. Focuses on large-scale data processing, data warehousing, and big data technologies.')

In [12]:
employee.description, employee.name

('Designs, builds, and maintains robust data pipelines and ETL processes. Focuses on large-scale data processing, data warehousing, and big data technologies.',
 'Kalle Johansson')

In [14]:
isinstance(employee, BaseModel)

True

In [16]:
employee.model_dump()

{'name': 'Kalle Johansson',
 'age': 35,
 'salary': 60000,
 'role': 'Senior Data Engineer',
 'description': 'Designs, builds, and maintains robust data pipelines and ETL processes. Focuses on large-scale data processing, data warehousing, and big data technologies.'}

In [18]:
employee.model_dump_json()

'{"name":"Kalle Johansson","age":35,"salary":60000,"role":"Senior Data Engineer","description":"Designs, builds, and maintains robust data pipelines and ETL processes. Focuses on large-scale data processing, data warehousing, and big data technologies."}'

In [20]:
print(employee.model_dump_json(indent=3))

{
   "name": "Kalle Johansson",
   "age": 35,
   "salary": 60000,
   "role": "Senior Data Engineer",
   "description": "Designs, builds, and maintains robust data pipelines and ETL processes. Focuses on large-scale data processing, data warehousing, and big data technologies."
}


simulate 10 employees

In [ ]:
result = await agent.run(
    "Give 10 employees in AI, data science, data engineering, data analyst field",
    output_type=list[Employee]
)

In [ ]:
for employee in result.output:
    print(
        f"{employee.name} is {employee.age} years, earns {employee.salary} and have role {employee.role}"
    )

Alice Johnson is 32 years, earns 65000 and have role Senior AI Engineer
Bob Williams is 45 years, earns 69000 and have role Expert Data Scientist
Charlie Brown is 38 years, earns 67000 and have role Senior Data Engineer
Diana Miller is 26 years, earns 45000 and have role Junior Data Analyst
Eve Davis is 40 years, earns 68000 and have role Senior AI Researcher
Frank White is 29 years, earns 55000 and have role Junior Machine Learning Engineer
Grace Lee is 35 years, earns 62000 and have role Senior Business Intelligence Analyst
Harry Green is 50 years, earns 69500 and have role Expert Data Architect
Ivy Hall is 33 years, earns 66000 and have role Senior Quantitative Analyst
Jack King is 28 years, earns 50000 and have role Junior Big Data Engineer


## More complex models

In [30]:
from typing import Literal

class Department(BaseModel):
    name: Literal["IT", "HR", "Marketing"] = Field(description="name of the department")
    description: str = Field(description="short description of the department")
    employees: list[Employee]

class Company(BaseModel):
    name: str = Field(description="name of the company")
    departments: list[Department]
    description: str = Field(description="describe company in two sentences")

result = await agent.run(
    ("Give 2-3 employees in each department",
     "Company should be called CoolCompany and sell ice creams"
     ),
    output_type=Company
)


In [32]:
company = result.output
company.__dict__.keys()

dict_keys(['name', 'departments', 'description'])

In [33]:
company.name

'CoolCompany'

In [35]:
[department.name for department in company.departments]

['IT', 'HR', 'Marketing']

In [38]:
departments = [department for department in company.departments]
departments[0].employees

[Employee(name='Alice Wonderland', age=30, salary=60000, role='Junior Software Developer', description='Responsible for developing and maintaining internal software tools. Works with Python and JavaScript.'),
 Employee(name='Bob The Builder', age=45, salary=65000, role='Senior IT Manager', description='Oversees the IT department operations and manages system architecture. Ensures smooth IT infrastructure.')]

In [40]:
company.model_dump()

{'name': 'CoolCompany',
 'departments': [{'name': 'IT',
   'description': 'Manages all technology infrastructure and software development for the company.',
   'employees': [{'name': 'Alice Wonderland',
     'age': 30,
     'salary': 60000,
     'role': 'Junior Software Developer',
     'description': 'Responsible for developing and maintaining internal software tools. Works with Python and JavaScript.'},
    {'name': 'Bob The Builder',
     'age': 45,
     'salary': 65000,
     'role': 'Senior IT Manager',
     'description': 'Oversees the IT department operations and manages system architecture. Ensures smooth IT infrastructure.'}]},
  {'name': 'HR',
   'description': 'Handles recruitment, employee relations, and company culture initiatives.',
   'employees': [{'name': 'Charlie Chaplin',
     'age': 35,
     'salary': 55000,
     'role': 'HR Specialist',
     'description': 'Manages employee onboarding, benefits administration, and supports HR policies. Focuses on employee well-being